In [2]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [34]:
folder = 'E:/Fragments/Vessels_200_obj_ply/'
look_for = '*it*.obj'
target_folder = 'E:/Fragments/Vessels_200_train_reduced/'
clean_target_folder = True
max_fractures = 5
max_models = 100
are_models_reduced = True

In [4]:
files = glob.glob(folder + '*/' + look_for)

In [14]:
print(len(files))

197600


In [15]:
if clean_target_folder:
    for file in glob.glob(target_folder + '/*'):
        os.remove(file)
        
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

PermissionError: [WinError 5] Acceso denegado: 'E:/Fragments/Vessels_200_train_reduced\\AL_03D_10f'

In [35]:
dataset_dict = {}

offset = 0
if are_models_reduced:
    offset = 1

for file in files:
    # split information from the file name
    file_info = file.split('\\')[-1].split('.')[0].split('_')
    fragment_id = file_info[-1 - offset]
    iteration = file_info[-2 - offset].split('it')[0]
    stitched_name = '_'.join(file_info[:-4 - offset])
    num_fragments = int(file_info[-4 - offset].split('f')[0])
    
    if stitched_name not in dataset_dict:
        dataset_dict[stitched_name] = { }
        
    if num_fragments not in dataset_dict[stitched_name]:
        dataset_dict[stitched_name][num_fragments] = {}
        
    if iteration not in dataset_dict[stitched_name][num_fragments]:
        dataset_dict[stitched_name][num_fragments][iteration] = []
        
    dataset_dict[stitched_name][num_fragments][iteration].append(file)
    
    # if len(dataset_dict) > max_models:
    #     break

In [ ]:
# from each object, select up to max_fractures iterations
sum_pieces = 0
folders = []

for stitched_name in tqdm(dataset_dict):
    folders.append(target_folder + stitched_name)
    
    # create the folder if it does not exist
    if not os.path.exists(target_folder + stitched_name):
        os.makedirs(target_folder + stitched_name)
        
    current_fractures = 0
    
    for num_fragments in dataset_dict[stitched_name]:
        # get the number of iterations
        iterations = list(dataset_dict[stitched_name][num_fragments].keys())
        iterations = [int(i) for i in iterations]
        iterations.sort()
        iterations = [str(i) for i in iterations]
        
        # select the iterations
        if len(iterations) > max_fractures:
            iterations = np.random.choice(iterations, max_fractures, replace=False)
            
        for iteration in iterations:
            sum_pieces += len(dataset_dict[stitched_name][num_fragments][iteration])
    
        # copy the files to the target folder
        for iteration in iterations:
            # create the iteration folder if it does not exist
            if not os.path.exists(target_folder + stitched_name + '/fractured_' + str(current_fractures)):
                os.makedirs(target_folder + stitched_name + '/fractured_' + str(current_fractures))

            for file_idx, file in enumerate(dataset_dict[stitched_name][num_fragments][iteration]):
                # copy the file to the target folder
                file = file.replace('/', '\\')
                os.system('copy ' + file + ' "' + target_folder + stitched_name + '/fractured_' + str(current_fractures) + '/piece_' + str(file_idx) + '.obj"')

            current_fractures += 1
    
print(sum_pieces)

 14%|█▎        | 27/200 [08:43<52:08, 18.09s/it]  

In [142]:
train_val_split = 0.8

np.random.shuffle(folders)
train_folders = folders[:int(len(folders) * train_val_split)]
val_folders = folders[int(len(folders) * train_val_split):]

with open(target_folder + 'artifact.train.txt', 'w') as f:
    for folder in train_folders:
        # folder file to local path
        folder = 'artifact/' + folder.split('E:/Fragments/Vessels_200_train/')[-1]
        f.write(folder + '\n')
        
with open(target_folder + 'artifact.val.txt', 'w') as f:
    for folder in val_folders:
        # folder file to local path
        folder = 'artifact/' + folder.split('E:/Fragments/Vessels_200_train/')[-1]
        f.write(folder + '\n')